<a href="https://colab.research.google.com/github/as9786/ParrotnlpJeju/blob/main/Jeju_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모듈 불러오기

In [ ]:
#모듈 불러오기
import pandas as pd
import numpy as np
import re
import codecs
import os
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-pjk1vh5l
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-pjk1vh5l
  Created wheel for pykospacing: filename=pykospacing-0.4-cp37-none-any.whl size=2255638 sha256=cc0d254cfcb7d7cecc94c135017b030838599fe230183bb82ec15b64d3ba0eed
  Stored in directory: /tmp/pip-ephem-wheel-cache-t05hqkj2/wheels/4d/45/58/e26cb2b7f6a063d234158c6fd1e5700f6e15b99d67154340ba
Successfully built pykospacing


In [ ]:
from pykospacing import spacing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 불러오기 및 전처리

In [ ]:
#경로 설정
PATH = '/content/drive/MyDrive/jejutxt'

In [ ]:
je = []
ko = []

In [ ]:
# 데이터 불러오기
for name in os.listdir(PATH):
  raw = codecs.open(f"{PATH}/{name}", 'r', 'utf8').read()
  sents = re.findall('[0-9]{6}.+[(][\S\s]+?[)][\n\r]+', raw) # 제주어-표준어 문장단위 추출
  # 페이지에 걸쳐 있는 문장 제외 / 
  sents = [sent for sent in sents if "구술" not in sent and not re.search(r'[\ue000-\uffff]', sent) and not re.search('[)][(]', sent) and re.search('[가-힣]+.+[(]', sent)] 
  je_tmp = [re.search('[가-힣]+.+[(]', sent).group()[:-1] for sent in sents]
  je_tmp = [re.sub('[\d()]', '', sent) for sent in je_tmp]
  ko_tmp = [re.search('[(].+[)]', sent, re.S).group()[1:-1] for sent in sents]
  je.extend(je_tmp)
  ko.extend(ko_tmp)

In [ ]:
# 이상치 데이터 확인
koerror = []
for sent in ko:
  if re.search('[()]',sent):
    koerror.append(sent)

In [ ]:
#이상치 data index 확인
error = []
for i in koerror:
  error.append(ko.index(i))

In [ ]:
# 이상치 data 제거
for i in error:
  del je[i]
  del ko[i]

In [ ]:
je[:20]

['동부락, 이 마을, 신촌 마을이 그냥 신촌? 여기 동부락이우꽈?',
 '이디 동동.',
 '동동? 동동예. 그 이 마을이 언제 형성뒌 건 알아지쿠과?',
 '몰라. 난.',
 '이 마을의 형성?',
 '옛날엔 숙군이옌 헤서 숙군. 숙군인디 이젠 신촌이옌 옛날엔 숙군.',
 '숙군이예?',
 '응.',
 '숙군이?',
 '신촌이옌 안헤영 숙군이렌 헷엇다고예?',
 '숙군이라고 이 마을이 숙어지니까 숙군.',
 '아아, 마을이 숙어지니까.',
 '이 신촌이 어디가도 보이질 안허여.',
 '아아.',
 '숙어지니까 숙군? 그믄 숙군이엇을 때는 조천에 들어가낫수과?',
 '조천읍에.',
 '조천읍 숙군리?',
 '조천 면 숙군리? 숙군리?',
 '숙군.',
 '숙군.']

In [ ]:
ko[:20]

['동부락,\n이 마을, 신촌 마을이 그냥 신촌? 여기 동부락입니까?',
 '여기 동동.',
 '동동? 동동이요.\n그 이 마을이 언제 형성된 건 알아지겠습니까?',
 '몰라. 나는.',
 '이 마을의 형성?',
 '옛날\n엔 ‘숙군’이라고 해서 ‘숙군’. ‘숙군’인데 이젠 신촌이라고 옛날엔 ‘숙군’.',
 '‘숙군’이요?',
 '응.',
 '‘숙군’이?',
 '신촌이라고 안하고 ‘숙군’이라고 했었다\n고요?',
 '‘숙군’이라고 이 마을이 숙어지니\n까 ‘숙군’.',
 '아아, 마을이 숙어지니까.',
 '이 신촌이 어디가도 보이질 않아.',
 '아아.',
 '숙어지니\n까 ‘숙군’? 그러면 ‘숙군’이었을 때는 조천에 들어갔었습니까?',
 '조천읍에.',
 '조천읍 숙군리?',
 '조천 면 ‘숙군리’? ‘숙군리’?',
 '‘숙군’',
 '‘숙군’.']

In [ ]:
print(len(je))
print(len(ko))

94044
94044


In [ ]:
# 공백문자 a로 변환
ko2 = [re.sub('[\n\r\t]','a',sent) for sent in ko]

In [ ]:
ko2[:10]

['동부락,a이 마을, 신촌 마을이 그냥 신촌? 여기 동부락입니까?',
 '여기 동동.',
 '동동? 동동이요.a그 이 마을이 언제 형성된 건 알아지겠습니까?',
 '몰라. 나는.',
 '이 마을의 형성?',
 '옛날a엔 ‘숙군’이라고 해서 ‘숙군’. ‘숙군’인데 이젠 신촌이라고 옛날엔 ‘숙군’.',
 '‘숙군’이요?',
 '응.',
 '‘숙군’이?',
 '신촌이라고 안하고 ‘숙군’이라고 했었다a고요?']

In [ ]:
# a가 포함되지 않은 표준어
ko4 = [sent for sent in ko2 if not re.search('[가-힣]*[a][가-힣]*',sent) ]

In [ ]:
ko4[:10]

['여기 동동.',
 '몰라. 나는.',
 '이 마을의 형성?',
 '‘숙군’이요?',
 '응.',
 '‘숙군’이?',
 '아아, 마을이 숙어지니까.',
 '이 신촌이 어디가도 보이질 않아.',
 '아아.',
 '조천읍에.']

In [ ]:
# 표준어 단어 사전 만들기
ko_word = word_tokenize(str(ko4))
ko_word[:10]

['[', "'여기", '동동', '.', "'", ',', "'몰라", '.', '나는', '.']

In [ ]:
#한국어를 제외한 문자열 제외
ko_word = [sent for sent in ko_word if not re.match('[^가-힣]',sent)]

In [ ]:
ko_word[:20]

['동동',
 '나는',
 '마을의',
 '형성',
 '숙군',
 '이요',
 '숙군',
 '이',
 '마을이',
 '숙어지니까',
 '신촌이',
 '어디가도',
 '보이질',
 '않아',
 '숙군리',
 '면',
 '숙군리',
 '숙군리',
 '숙군',
 '숙군']

In [ ]:
# 표준어 단어 사전
kovoca = set(ko_word)
len(kovoca)

21982

In [ ]:
# a 붙어있는 데이터 전처리
kof = []
for sent in ko2:
  if re.search('[가-힣]*[a]+[가-힣]*',sent):
    a = re.search('[가-힣]*[a]+[가-힣]*',sent).group()
    b = re.sub('a','',a)
    if b in kovoca: #ko단어 사전에 있을 경우 단어 사전의 단어로 변환
      c = re.sub('a','',sent)
      kof.append(c)
    else: # ko단어 사전에 없는 경우 spacing 단어 사전의 단어로 변환
      d = re.sub('a','',sent)
      kof.append(spacing(d)) 
  else:
    kof.append(sent)  

In [ ]:
kof[:20]

['동부락,이 마을, 신촌 마을이 그냥 신촌? 여기 동부락입니까?',
 '여기 동동.',
 '동동? 동동이요.그 이 마을이 언제 형성된 건 알아지겠습니까?',
 '몰라. 나는.',
 '이 마을의 형성?',
 '옛날엔 ‘숙군’이라고 해서 ‘숙군’. ‘숙군’인데 이젠 신촌이라고 옛날엔 ‘숙군’.',
 '‘숙군’이요?',
 '응.',
 '‘숙군’이?',
 '신촌이라고 안하고 ‘숙군’이라고 했었다고요?',
 '‘숙군’이라고 이 마을이 숙어지니까 ‘숙군’.',
 '아아, 마을이 숙어지니까.',
 '이 신촌이 어디가도 보이질 않아.',
 '아아.',
 '숙어지니까 ‘숙군’? 그러면 ‘숙군’이었을 때는 조천에 들어갔었습니까?',
 '조천읍에.',
 '조천읍 숙군리?',
 '조천 면 ‘숙군리’? ‘숙군리’?',
 '‘숙군’',
 '‘숙군’.']

In [ ]:
len(kof)

94044